In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Importação

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')

im=im[im.CO_ANO!=2024]
im.KG_LIQUIDO=im.KG_LIQUIDO/10**6
im=im.rename(columns={'KG_LIQUIDO':'kton'})
im.head(1)

,CO_ANO,CO_MES,kton
0,1997,3,7.505776


In [3]:
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
ch=ch[ch.CO_ANO!=2001]
ch.tail(1)

,CO_ANO,CO_MES,mm
264,2023,12,78.6


In [ ]:
q=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv')

In [5]:
for md in [None,1]:
    
    from sklearn.ensemble import RandomForestRegressor as RFR
    RFR=RFR(max_depth=md)
    
    for ad in [7,8]:
    

        i=im.copy()
        c=ch.copy()
    
        c.CO_MES=c.CO_MES+ad
        for adi in range(1,ad+1):
            c.loc[c.CO_MES==12+adi,'CO_ANO']=c.loc[c.CO_MES==12+adi]['CO_ANO']+1
            c.loc[c.CO_MES==12+adi,'CO_MES']=adi
        
        i=i.merge(c)

        i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1').astype(int)
        i=i.drop(columns='CO_ANO')
        i.CO_MES=i.CO_MES.astype('category')
        i=pd.get_dummies(i)
        
        for a in range(1,len(i)):
    
        
            for train_size_mes in [5*12,8*12]:

    
                train=i.iloc[:a,:].tail(train_size_mes)
                
                if len(train)>=train_size_mes:

    
                    
                    test=i.iloc[a:a+1,:]
                                
                    # Train
                    X_train_mm=train.drop(columns='kton')
                    X_train_s_mm=train.drop(columns=['kton','mm'])
                    y_train=train.kton
            
                    # Test
                    X_test_mm  =test.drop(columns='kton')
                    X_test_s_mm=test.drop(columns=['kton','mm'])
                    y_test=test.kton
                        
                    RFR.fit(X_train_mm,y_train)
                    
                    test_pred=test     .assign(kton_pred_mm  =RFR.predict(X_test_mm  ).astype(int))
                    
                    RFR.fit(X_train_s_mm,y_train)
                    
                    test_pred=test_pred.assign(kton_pred_s_mm=RFR.predict(X_test_s_mm).astype(int))
    
                    test_pred=test_pred.assign(ad=ad,train_size_anos=train_size_mes/12,md=md)
                    
                    q=pd.concat([q,test_pred])

q['erro_mm']=np.abs(q.kton-q.kton_pred_mm)
q['erro_s_mm']=np.abs(q.kton-q.kton_pred_s_mm)

print('erro_mm.mean',round(q.erro_mm.mean()))
print('erro_s_mm.mean',round(q.erro_s_mm.mean()))

erro_mm.mean 36
erro_s_mm.mean 39


In [6]:
q

,kton,mm,date,CO_MES_1,CO_MES_2,CO_MES_3,CO_MES_4,CO_MES_5,CO_MES_6,CO_MES_7,...,CO_MES_10,CO_MES_11,CO_MES_12,kton_pred_mm,kton_pred_s_mm,ad,train_size_anos,md,erro_mm,erro_s_mm
60,53.557361,0.4,1185926400000000000,False,False,False,False,False,False,False,...,False,False,False,48,61,7,5.0,None,5.557361,7.442639
61,50.424205,0.3,1188604800000000000,False,False,False,False,False,False,False,...,False,False,False,57,63,7,5.0,None,6.575795,12.575795
62,79.445819,0.0,1191196800000000000,False,False,False,False,False,False,False,...,True,False,False,42,57,7,5.0,None,37.445819,22.445819
63,56.522721,8.8,1193875200000000000,False,False,False,False,False,False,False,...,False,True,False,69,65,7,5.0,None,12.477279,8.477279
64,40.785987,0.4,1196467200000000000,False,False,False,False,False,False,False,...,False,False,True,57,59,7,5.0,None,16.214013,18.214013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,240.914358,19.9,1696118400000000000,False,False,False,False,False,False,False,...,True,False,False,133,136,8,8.0,1,107.914358,104.914358
254,90.119779,83.2,1698796800000000000,False,False,False,False,False,False,False,...,False,True,False,163,178,8,5.0,1,72.880221,87.880221
254,90.119779,83.2,1698796800000000000,False,False,False,False,False,False,False,...,False,True,False,141,141,8,8.0,1,50.880221,50.880221
255,128.496690,50.5,1701388800000000000,False,False,False,False,False,False,False,...,False,False,True,145,151,8,5.0,1,16.503310,22.503310


In [7]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv', index=False)